# Red Neuronal en Keras (basada en notebook PyTorch)


Este notebook implementa una red neuronal en **Keras** equivalente a la versión en PyTorch usada en la Tarea 11.  
Se mantiene el mismo preprocesamiento (`preprocessing.py`) y estructura general del flujo de entrenamiento.

**Arquitectura**: 256 → 128 → 1 con `ReLU`, Dropout y EarlyStopping.  
Se incluye evaluación del RMSE sobre el conjunto de test.


In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from preprocessing import get_preprocessor
import tensorflow as tf
from tensorflow import keras


In [ ]:

# Cargar datos
housing = pd.read_csv("housing.csv").dropna()
X = housing.drop("median_house_value", axis=1)
y = housing["median_house_value"]

# División
X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, test_size=0.2, random_state=42)

# Escalado del target
y_scaler = StandardScaler()
y_train_scaled = y_scaler.fit_transform(y_train.values.reshape(-1, 1))
y_valid_scaled = y_scaler.transform(y_valid.values.reshape(-1, 1))
y_test_scaled = y_scaler.transform(y_test.values.reshape(-1, 1))

# Preprocesamiento de features
preprocessor = get_preprocessor(housing)
X_train_prep = preprocessor.fit_transform(X_train)
X_valid_prep = preprocessor.transform(X_valid)
X_test_prep = preprocessor.transform(X_test)


In [ ]:

# Arquitectura: 256 → 128 → 1
model = keras.models.Sequential([
    keras.layers.Input(shape=X_train_prep.shape[1]),
    keras.layers.Dense(256, activation="relu"),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dense(1)
])

model.compile(loss="mean_squared_error", optimizer=keras.optimizers.Adam(learning_rate=0.001))
model.summary()


In [ ]:

early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)

history = model.fit(
    X_train_prep, y_train_scaled,
    epochs=100,
    validation_data=(X_valid_prep, y_valid_scaled),
    callbacks=[early_stopping_cb]
)


In [ ]:

plt.plot(history.history["loss"], label="train")
plt.plot(history.history["val_loss"], label="val")
plt.title("Evolución del error (MSE)")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.grid(True)
plt.legend()
plt.show()


In [ ]:

y_test_pred_scaled = model.predict(X_test_prep)
y_test_pred = y_scaler.inverse_transform(y_test_pred_scaled)
rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))
print("✅ RMSE test (modelo Keras):", rmse_test)

plt.figure(figsize=(8,6))
plt.scatter(y_test, y_test_pred, alpha=0.4)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], "r--")
plt.xlabel("Valor real ($)")
plt.ylabel("Predicción ($)")
plt.title("Red neuronal Keras vs valores reales (test)")
plt.grid(True)
plt.show()
